In [1]:
import pandas as pd
import numpy as np
import os
import joblib

# Bibliotecas de Google Colab y Kaggle
from google.colab import drive
import kagglehub

# Bibliotecas de Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import recall_score, precision_score, classification_report


# --- 1. CONFIGURACIÓN Y DESCARGA DE DATOS ---
print("--- 1. CONFIGURACIÓN E INICIO DE LA DESCARGA ---")

# Montar Google Drive
drive.mount('/content/drive')

# Definir y crear la ruta de guardado en Drive
DRIVE_PATH = '/content/drive/MyDrive/Modelos_Perceptron_Fraude/'
if not os.path.exists(DRIVE_PATH):
    os.makedirs(DRIVE_PATH)
    print(f"Carpeta creada en Drive: {DRIVE_PATH}")
else:
    print(f"Carpeta de Drive ya existe: {DRIVE_PATH}")

# Descargar el dataset de Kaggle
# Asegúrese de que sus credenciales de Kaggle estén configuradas en Colab (API token).
try:
    path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
    file_path = os.path.join(path, 'creditcard.csv')
    data = pd.read_csv(file_path)
    print(f"✅ Dataset descargado y cargado desde: {file_path}")
except Exception as e:
    print(f"❌ ERROR al descargar o cargar el dataset. Asegúrese de tener el token de Kaggle configurado.")
    print(f"Detalle del error: {e}")
    # Detener ejecución si falla la carga
    exit()

print("Forma del Dataset:", data.shape)


# --- 2. PRE-PROCESAMIENTO DE DATOS Y DIVISIÓN ---
print("\n--- 2. PRE-PROCESAMIENTO ---")

# Definición de la Variable Objetivo y Features
Y = data['Class']
X_all = data.drop('Class', axis=1)

# Escalado de Features (Crucial para el Perceptrón)
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)
X_all_scaled = pd.DataFrame(X_all_scaled, columns=X_all.columns)
print("Features escaladas correctamente (StandardScaler).")

# División de datos (Entrenamiento y Prueba), estratificado para manejar el desbalance
X_train_all, X_test_all, Y_train, Y_test = train_test_split(
    X_all_scaled, Y, test_size=0.3, random_state=42, stratify=Y
)
print(f"Datos divididos: Entrenamiento={X_train_all.shape[0]}, Prueba={X_test_all.shape[0]}")


# --- 3. DEFINICIÓN DE CONJUNTOS DE FEATURES PARA EL EXPERIMENTO ---

# Conjunto A: BASE (Todas las Features)
FEATURES_A = X_all_scaled.columns.tolist()

# Conjunto B: Modelo 3 - Menor Dimensionalidad (V1-V14 + Time/Amount)
FEATURES_B = ['Time', 'Amount'] + [f'V{i}' for i in range(1, 15)]

# Conjunto C: Modelo 3 - Features Clave (Basado en relevancia común)
FEATURES_C = ['Time', 'Amount', 'V4', 'V11', 'V12', 'V14', 'V16', 'V17', 'V18']


# --- 4. FUNCIÓN DE ENTRENAMIENTO, EVALUACIÓN Y GUARDADO ---

def train_save_and_evaluate_perceptron(X_train, X_test, Y_train, Y_test, features_list, model_name, learning_rate=0.01):
    """
    Entrena, evalúa y GUARDA el modelo Perceptrón en Google Drive.
    """

    print(f"\n--- {model_name} (Features: {len(features_list)}) ---")

    # 1. Preparar datos filtrando las features seleccionadas
    X_train_filtered = X_train[features_list]
    X_test_filtered = X_test[features_list]

    # 2. Inicializar y Entrenar Perceptrón
    # eta0 es la tasa de aprendizaje, max_iter son las épocas
    model = Perceptron(
        eta0=learning_rate,
        max_iter=50,
        random_state=42,
        shuffle=True,
        n_jobs=-1,
        tol=1e-3
    )
    model.fit(X_train_filtered, Y_train)

    # 3. Guardar el modelo en Google Drive
    model_filename = f"{model_name.replace(' ', '_')}_LR{str(learning_rate).replace('.', 'p')}.joblib"
    save_path = os.path.join(DRIVE_PATH, model_filename)
    joblib.dump(model, save_path)
    print(f"✅ Modelo GUARDADO en Google Drive en: {save_path}")

    # 4. Evaluación
    Y_pred = model.predict(X_test_filtered)

    # Métricas clave para el desbalance: Recall y Precisión para la clase 1 (Fraude)
    recall = recall_score(Y_test, Y_pred)
    precision = precision_score(Y_test, Y_pred)

    print(f"Features utilizadas: {', '.join(features_list[:min(len(features_list), 5)])}...")
    print(f"Tasa de Aprendizaje (eta0): {learning_rate}")
    print(f"Recall (Clase 1 - Fraude): {recall:.4f}")
    print(f"Precisión (Clase 1 - Fraude): {precision:.4f}")

    # Mostrar reporte completo
    print("\nReporte de Clasificación:")
    print(classification_report(Y_test, Y_pred, target_names=['Legítimo (0)', 'Fraude (1)']))


# --- 5. EJECUCIÓN DE LOS 3 MODELOS Y ANÁLISIS DEL IMPACTO DE LAS FEATURES ---

print("\n\n#################################################################")
print("## INICIO DE LOS EXPERIMENTOS DEL PERCEPTRÓN 3: VARIACIÓN DE FEATURES")
print("#################################################################")

# 1. MODELO BASE (Perceptrón 1/2) - Usando todas las features
train_save_and_evaluate_perceptron(
    X_train_all, X_test_all, Y_train, Y_test,
    features_list=FEATURES_A,
    model_name="MODELO_BASE_FEATURES_A",
    learning_rate=0.01
)

# 2. PERCEPTRÓN 3 - VARIACIÓN B (Menor Dimensionalidad)
# ÉNFASIS: ¿Ayuda la eliminación de features "menos importantes"?
train_save_and_evaluate_perceptron(
    X_train_all, X_test_all, Y_train, Y_test,
    features_list=FEATURES_B,
    model_name="PERCEPTRON_3_VARIACION_B",
    learning_rate=0.01
)

# 3. PERCEPTRÓN 3 - VARIACIÓN C (Features Clave)
# ÉNFASIS: ¿Un subconjunto de features altamente discriminatorias mejora el rendimiento lineal?
train_save_and_evaluate_perceptron(
    X_train_all, X_test_all, Y_train, Y_test,
    features_list=FEATURES_C,
    model_name="PERCEPTRON_3_VARIACION_C",
    learning_rate=0.01
)

print("\n\n#################################################################")
print("## ¡EXPERIMENTO FINALIZADO!")
print(f"Los modelos .joblib han sido guardados en su Drive en: {DRIVE_PATH}")
print("#################################################################")

--- 1. CONFIGURACIÓN E INICIO DE LA DESCARGA ---
Mounted at /content/drive
Carpeta creada en Drive: /content/drive/MyDrive/Modelos_Perceptron_Fraude/
Using Colab cache for faster access to the 'creditcardfraud' dataset.
✅ Dataset descargado y cargado desde: /kaggle/input/creditcardfraud/creditcard.csv
Forma del Dataset: (284807, 31)

--- 2. PRE-PROCESAMIENTO ---
Features escaladas correctamente (StandardScaler).
Datos divididos: Entrenamiento=199364, Prueba=85443


#################################################################
## INICIO DE LOS EXPERIMENTOS DEL PERCEPTRÓN 3: VARIACIÓN DE FEATURES
#################################################################

--- MODELO_BASE_FEATURES_A (Features: 30) ---
✅ Modelo GUARDADO en Google Drive en: /content/drive/MyDrive/Modelos_Perceptron_Fraude/MODELO_BASE_FEATURES_A_LR0p01.joblib
Features utilizadas: Time, V1, V2, V3, V4...
Tasa de Aprendizaje (eta0): 0.01
Recall (Clase 1 - Fraude): 0.7703
Precisión (Clase 1 - Fraude): 0.7037

Reporte